In this notebook all the steps described in the report are run with the Genetic Algorithm. Execution of the full notebook can take a few hours, so that we have saved the outputs of the algorithms as text files that can be loaded instead of running the algorithms again.

The lines of code used to run the algorithms and saving their outputs are commented out.

### Note about distance functions:

In [1]:
from datetime import datetime
import json
import numpy as np

In [2]:
cities = np.genfromtxt('cities.csv', delimiter=',', skip_header = 1)

In [3]:
from santas_path import edp, edp_unordered_straight, total_length_straight, total_length_loop

Many distance functions are used in this notebook to compute the length of a tour. We briefly describe their use.

The function "edp", short for Euclidean Distance with Penalties, is the most important distance function in this notebook. It computes the total length of a route through the cities as the Euclidean distance between each step, but each 10th step originating from a non-prime city suffers a 10% penalty. Its implementation takes on imput a route, an array of city ids and their coordinates, and a black list: the city ids related to 10th steps will be compared to this list in order to decide if they should be penalized. In our case, the black list will be a boolean mask, with 1s corresponding to non-prime numbers. The function returns the total length of the route, starting and ending at the city with ID 0. Its implementation assumes that the city ids in the first column of the cities array passed as input correspond to the row index of the city. E.g., the city with ID 0 has index 0, the city with ID 1 has index 1, and so on. If this is not the case, the next function should be used. 

The function "edp_unordered_straight" is the same as edp, except that it does not assume that the cities array in input is well structured as described earlier. Thus, it is useful to apply to inner segments of a route and subsest of the cities dataset. It will be used in the very final step, when SA is applied (see report). It is called "straight" because the tour whose distance is computed by the function need not start and end at the same city (in case this is required, the santas_path library also contains a "edp_unordered_loop" function). 

The functions "total_length_straight" and "total_length_loop" compute the distance of a route through the cities, without penalising 10th steps originating from cities with a non-prime ID. The first function computes the distance on routes starting and ending at different cities, while the second one on routes starting and ending at the same city.

Note: throughout the notebook, the functions used to compute the distance of a route will take care of adding 0 at the start or at the and of the route during computation, if needed. For this reason, some "full" routes in this notebook might not start with 0, or not end with 0, but it will not make a difference. The function "edp" was tested on routes from the Kaggle challenge leaderboard in order to make sure of its correctness.

# Clustering

We partition the cities in 1000 clusters using k-means.

In [4]:
n = 1000

We have run the following code and saved the clusters as a txt file, so we can just import it now

In [5]:
# from sklearn.cluster import KMeans
# kmeans = KMeans(n_clusters=n, random_state=0)
# start = datetime.now()
# kmeans.fit(cities[:, 1:3])
# end = datetime.now()
# (end-start).total_seconds() / 60
# np.savetxt('files/1000subsets.txt', kmeans.labels_) 

In [6]:
clusters = np.genfromtxt('files/1000subsets.txt', skip_header = 0).astype(int)

In [7]:
clusters

array([894, 654, 614, ..., 263, 305,  80])

We add the information regarding the cluster to the cities array.

In [8]:
kcities = np.concatenate((cities, clusters[:, np.newaxis]), 1)

In [9]:
kcities[:5]

array([[0.00000000e+00, 3.16836739e+02, 2.20234071e+03, 8.94000000e+02],
       [1.00000000e+00, 4.37740597e+03, 3.36602082e+02, 6.54000000e+02],
       [2.00000000e+00, 3.45415820e+03, 2.82005301e+03, 6.14000000e+02],
       [3.00000000e+00, 4.68809930e+03, 2.93589806e+03, 5.26000000e+02],
       [4.00000000e+00, 1.01069695e+03, 3.23675099e+03, 1.90000000e+01]])

We create a list of clusters ("subsets")

In [10]:
subs = [0]*n
for i in range(n):
    subs[i] = kcities[kcities[:, 3] == i][:, :3]

In [11]:
subs = np.array(subs)

First rows of the cluster 0:

In [12]:
subs[0][:5]

array([[ 363.        ,  359.17708865, 2894.43630611],
       [ 649.        ,  404.28160286, 2897.62661906],
       [1305.        ,  421.01277961, 2886.92349504],
       [1890.        ,  444.20103667, 2908.49426528],
       [9018.        ,  419.39723055, 2864.27279966]])

The following are the sizes of some of the clusters:

In [13]:
lens = np.array([len(subs[i]) for i in range(n)])
lens[:50]

array([230, 118, 121, 168,  89, 163, 310, 288, 277, 156, 289, 228, 172,
       273, 151, 156, 261, 236, 190, 234, 183,  73, 242, 222, 228, 161,
       244, 282, 265, 198, 138, 288, 268, 181, 274, 158, 160, 203, 127,
       152, 216, 313, 222, 192, 218, 145, 241, 296, 151, 137])

The smallest cluster has only 12 cities:

In [14]:
np.min(lens)

12

The largest cluster consists of 407 cities:

In [15]:
np.max(lens)

407

The median cluster size is 201:

In [16]:
np.median(lens)

201.0

The size of the union of the clusters is 197769, as expected.

In [17]:
sum(lens)

197769

# Running GA in each cluster

In [18]:
from ga import GA, route_fitness, shift_mutation, roulette_selection, two_point_crossover, pop_gen

For later comparison, we compute the average total length and average edp of a random route through all the cities:

In [19]:
# generate 100 random routes
np.random.seed(4)
random_routes = pop_gen(len(cities), 100, include_zero=True)

Average total distance of a random route:

In [20]:
rand_lengths = np.apply_along_axis(total_length_loop, 1, random_routes, c = cities)
np.mean(rand_lengths)

442919388.5838148

To calculate the edp distance, we create a boolean mask  for non-prime number: every city_id such that "not_primes_bool[city_id] = True" will be penalized.

In [21]:
from santas_path import not_prime

In [22]:
np_not_prime = np.vectorize(not_prime)
nums = np.arange(0, len(cities))
not_primes_bool = np_not_prime(nums)

Average edp distance of a random route:

In [23]:
np.random.seed(4)
random_routes = pop_gen(len(cities), 100, include_zero=False)

In [24]:
rand_edps = np.apply_along_axis(edp, 1, random_routes, c = cities, black_list = not_primes_bool)
np.mean(rand_edps)

447106962.3408601

We now initialize the permutations in each cluster. We want to find the best permutation of cities inside each cluster, where a permutation inside a cluster is a permutation of the range of the size of the cluster, i.e., it is an ordering of cities.

In [25]:
subs_perms = [0]*n
for i in range(n):
    subs_perms[i] = np.arange(len(subs[i]))

We create a list of lists, each consisting of the city ids in each cluster:

In [26]:
c_ids = [el[:,0] for el in subs]

We can find the route inside each cluster (as a sequence of city ids) as follows:

In [27]:
start_routes = []
for i in range(n):
    start_routes.append(c_ids[i][subs_perms[i]])

In order to make a comparison with the total distance of the route that we will soon find by using GA to find an optimal permutation inside each cluster, we concatenate all the routes inside each cluster as they are at the moment, that is, just ordered by their city id.

In [28]:
start_route = np.concatenate(start_routes).astype(int)

The total length of the route obtained by clustering the cities, but without ordering the cities within each cluster, or without ordering the clusters, is the following, which is already a significant improvement over a random path.

In [29]:
total_length_loop(start_route, cities)

13139371.401815962

We now run GA inside each cluster in order to find the best permutation of cities in each "province". As described in the report, at this stage we treat this problem as a standard TSP problem.

In [30]:
# np.random.seed(4)
# start = datetime.now()
# startl = datetime.now()
# p = 0 
# initial_dist = total_length_loop(start_route, cities)
# last_dist = initial_dist
# print('Initial distance: {}'.format(last_dist))
# for i in range(n):
#     if i % 100 == 0:
#         print('\nStart Loop {} at {}'.format(i, startl))
#     subs_perms[i] = GA(subs[i], np.inf, 30, 10, route_fitness, [shift_mutation], 0.1,
#                                       sel_fun=roulette_selection, cross_fun=two_point_crossover,
#                                       max_no_change = 500, length_fun = total_length_straight)
#     if (i % (99 + p) == 0) and i > 0:
#         p += 100
#         endl = datetime.now()
#         print('End loop {} at {}: {} seconds'.format(i, endl, (endl-startl).total_seconds()))
#         startl = datetime.now()
#         temp_routes = []
#         for j in range(n):
#             temp_routes.append(c_ids[j][subs_perms[j]])
#         temp_full_route = np.concatenate(temp_routes).astype(int)
#         temp_tot = total_length_loop(temp_full_route, cities)
#         print('Total distance so far: {}'.format(temp_tot))
#         if all(np.isin(temp_full_route, cities[:,0])) and all(np.isin(cities[:,0], temp_full_route)):
#             print("Checked: the path goes through all cities")
#         print('Improvement: {}'.format(temp_tot - last_dist))
#         print('Total Improvement: {}'.format(temp_tot - initial_dist))
#         last_dist = temp_tot    
#         
# end = datetime.now()
# print('\nTotal time: {}'.format((end-start).total_seconds()))

Running the algorithm we have updated the list 'subs_perms' of permutations inside each cluster. Each element subs_perms[i] in this list is the best permutation found by running GA on subs[i]. We now order each list c_ids[i] by subs_perms[i] in order to obtain the routes as specified by the city ids.

In [31]:
# routes = []
# for i in range(n):
#     routes.append(c_ids[i][subs_perms[i]])

We can now concatenate these routes in order to find a route on the full set of cities.

In [32]:
# full_route = np.concatenate(routes)

We order the route so that it starts and ends at 0.

In [33]:
# zi = np.where(full_route == 0)[0][0]

In [34]:
# full_route = np.concatenate((full_route[zi:], full_route[:zi]))

In [35]:
# full_route = full_route.astype(int)

We save this route so that we do not have to run the algorithm again:

In [36]:
# np.savetxt('files/full_route_GA_in_clusts.txt', full_route)

We load the saved file containing this route:

In [37]:
full_route = np.genfromtxt('files/full_route_GA_in_clusts.txt').astype(int)

In [38]:
cities = np.genfromtxt('cities.csv', delimiter=',', skip_header = 1)

As a check, we can see that the length of the full route is correct.

In [39]:
len(full_route)

197769

Also, every city in the path is the id of a city:

In [40]:
all(np.isin(full_route, cities[:,0])) 

True

... and every city id is in the path:

In [41]:
all(np.isin(cities[:,0], full_route)) 

True

The total length through this route:

In [42]:
total_length_loop(full_route, cities)

7805290.363298486

Finding the best permutation of cities inside each cluster has improved the total length of the route from ~13 139 371 to ~7 805 290

Let us now see what would be the length of this route if we penalized every 10th step not starting from a prime city:

In [43]:
edp(full_route, cities, not_primes_bool)

7878150.9519051965

For comparison, the edp distance of the start_route, before finding a better route in each cluster, is the following:

In [44]:
edp(start_route, cities, not_primes_bool)

13261882.759004043

#### Save subroutes as json

We also save the subroutes as a json file:

In [45]:
# routes_dict = dict()
# for i in range(len(routes)):
#     routes_dict[i] = list(routes[i])

In [46]:
# with open('files/subroutes_GA_in_clusts.json', 'w') as fp:
#     json.dump(routes_dict, fp)

# Running GA to find order of subroutes

We now have 1000 clusters and an optimal route inside each cluster. Now we want to apply GA to find the best way to order these 1000 clusters.

We reload the routes saved in the previous step:

In [47]:
with open('files/subroutes_GA_in_clusts.json', 'r') as fp:
    loaded_json = json.load(fp)

In [48]:
routes = [loaded_json[str(i)] for i in range(1000)]

In [49]:
for i in range(len(routes)):
    routes[i] = np.array(routes[i]).astype(int)

In [50]:
routes = np.array(routes)

We now apply GA for ordering the clusters.

In [51]:
from ga import subset_fitness

In [52]:
# np.random.seed(4)
# start = datetime.now()
# clusts_perm = GA(cities, np.inf, 30, 10, subset_fitness, [shift_mutation], 0.1,
#                         roulette_selection, cross_fun=two_point_crossover, max_no_change = 500,
#                         length_fun = total_length_loop, on_subsets= True, subs = routes, verbose = True)
# end = datetime.now()
# (end-start).total_seconds() / 60

We have obtained clusts_perm, which is the order of clusters

In [53]:
# new_full_route = np.concatenate(routes[clusts_perm])

We order it so that it starts and ends at 0:

In [54]:
# zin = np.where(new_full_route == 0)[0][0]
# new_full_route = np.concatenate((new_full_route[zin:], new_full_route[:zin]))

We save the route so we do not need to run the algorithm again:

In [55]:
# np.savetxt('files/route_after_clust_ordering.txt', new_full_route)

We load the route:

In [56]:
new_full_route = np.genfromtxt('files/route_after_clust_ordering.txt').astype(int)

In [57]:
cities = np.genfromtxt('cities.csv', delimiter=',', skip_header = 1)

In [58]:
new_full_route

array([     0, 161041,  97580, ...,  25100, 195889,  69414])

The total length through this route, starting and ending at 0, is the following:

In [59]:
total_length_loop(new_full_route, cities)

6726045.709191153

We permform the usual checks.
Every city in the path is the id of a city...

In [60]:
all(np.isin(new_full_route, cities[:,0]))

True

... and every city is in the path

In [61]:
all(np.isin(cities[:,0], new_full_route))

True

The total distance considering penalties is not very different:

In [62]:
edp(new_full_route[1:], cities, not_primes_bool)

6788202.383675191

# Improving the route by moving prime cities

We start from the route found in the previous step with length 6 788 202.383675191

Reload route:

In [63]:
start_route = np.genfromtxt('files/route_after_clust_ordering.txt').astype(int)

In [64]:
edp(start_route, cities, not_primes_bool)

6788202.383675191

We sort the cities dataset by the route found in the previous step. 

In [65]:
sorted_cities = cities[start_route[1:]]

We have excluded the first city in the route (i.e., the city with ID 0), as we want the SA to only work on inner segments of the full route.

We now want to run Simulated Annealing, starting from this route, with a special mutation function that reverses the order of two consecutive cities $c_1$ and $c_2$ with very high probability if $c_1$ is not prime and found at a tenth step, and $c_2$ is prime. We divide the route and the cities dataset in 1000 subsets so that SA can focus on a segment of the route at a time.

We create 1000 new subsets, each of size 200 (except from the last). 

In [66]:
clusters = np.repeat(np.arange(1000), 200)[:len(cities)-1]
len(clusters)

197768

We add the information about membership in each subset to the cities dataset sorted by the route:

In [67]:
kscities = np.concatenate((sorted_cities, clusters[:, np.newaxis]), 1)

In [68]:
kscities[:5]

array([[161041.        ,    333.4324635 ,   2215.18812375,
             0.        ],
       [ 97580.        ,    326.53583468,   2200.95264142,
             0.        ],
       [  2832.        ,    336.74566244,   2163.50537169,
             0.        ],
       [ 17097.        ,    331.77859752,   2148.64371451,
             0.        ],
       [174392.        ,    306.24676402,   2108.66894447,
             0.        ]])

There are 989 unique clusters

In [69]:
n = len(np.unique(clusters))
n

989

We create the list of clusters ("subsets"):

In [70]:
subs = [0]*n

In [71]:
for i in range(n):
    subs[i] = kscities[kscities[:, 3] == i][:, :3]

The first rows of the first subset:

In [72]:
subs[0][:5]

array([[161041.        ,    333.4324635 ,   2215.18812375],
       [ 97580.        ,    326.53583468,   2200.95264142],
       [  2832.        ,    336.74566244,   2163.50537169],
       [ 17097.        ,    331.77859752,   2148.64371451],
       [174392.        ,    306.24676402,   2108.66894447]])

In [73]:
lens = [len(subs[i]) for i in range(n)]
np.array(lens[-20:])

array([200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
       200, 200, 200, 200, 200, 200, 168])

We initialize the permutation on each subset:

In [74]:
subs_perm = [0]*n

In [75]:
for i in range(n):
    subs_perm[i] = np.arange(len(subs[i]))

In [76]:
c_ids = [el[:,0] for el in subs]

We run SA on each subset, with the described mutation function.

In [77]:
from sa import SA, reverse_primes_mutation
from santas_path import edp_unordered_straight

In [78]:
# np.random.seed(4)
# start = datetime.now()
# startl = datetime.now()
# p = 0
# last_dist = edp(start_route, cities, not_primes_bool)
# initial_dist = last_dist
# print('Initial distance: {}'.format(last_dist))
# for i in range(n):
#     if (i % 100 == 0) and (i > 0):
#         print('\nStart Loop {} at {}'.format(i, startl))
#     
#     perm_init = np.arange(len(subs[i]))
#     subs_perm[i] = SA(subs[i], edp_unordered_straight, reverse_primes_mutation,
#                           black_list = not_primes_bool, scale = 100000, n_to_mute= 10,
#                           maxIter = np.inf, perm_init = perm_init, maxIterNoChange=1000)
#     if ((i % (99 + p) == 0) or (i == n - 1)) and i > 0:
#         p += 100
#         endl = datetime.now()
#         print('End loop {} at {}: {} seconds'.format(i, endl, (endl-startl).total_seconds()))
#         startl = datetime.now()
#     if i % 20 == 0:
#         temp_routes = []
#         for j in range(n):
#             temp_routes.append(c_ids[j][subs_perm[j]])
#         temp_full_route = np.concatenate(temp_routes).astype(int)
#         temp_full_route = np.concatenate(([0], temp_full_route))
#         temp_tot = edp(temp_full_route[1:], cities, not_primes_bool)
#         print('\nLoop {}, Total distance so far: {}'.format(i, temp_tot))   
#         if all(np.isin(temp_full_route, cities[:,0])) and all(np.isin(cities[:,0], temp_full_route)):
#             print("Checked: the path goes through all cities")
#         print('Improvement: {}'.format(temp_tot - last_dist))
#         print('Total Improvement: {}'.format(temp_tot - initial_dist))
#         last_dist = temp_tot        
# end = datetime.now()
# print('\nTotal time: {}'.format((end-start).total_seconds()))

Similarly as before, we find the route inside each cluster as specified by the cities ids, and we concatenate it (and add the first city with id 0 at the start) to find the final route.

In [79]:
# c_ids = [el[:,0] for el in subs]
# routes = []
# for i in range(n):
#     routes.append(c_ids[i][subs_perm[i]])
# 
# final_full_route = np.concatenate(routes).astype(int)
# final_full_route = np.concatenate(([0], final_full_route))

In [80]:
# np.savetxt('files/final_after_sa.txt', final_full_route)

We have thus obtained the following route:

In [81]:
final_full_route = np.genfromtxt('files/final_after_sa.txt').astype(int)

In [82]:
cities = np.genfromtxt('cities.csv', delimiter=',', skip_header = 1)

In [83]:
final_full_route

array([     0, 161041,  97580, ..., 144501, 195889,  69414])

In [84]:
total_length_loop(final_full_route, cities)

6618859.716599304

In [85]:
edp(final_full_route, cities, not_primes_bool)

6679052.190203142

The edp of the final route is 6 679 052.190203142

# No Clustering 

In [86]:
np.random.seed(4)
start = datetime.now()
no_clust_route = GA(cities, 6000, 20, 10, route_fitness, [shift_mutation], 0.1,
              sel_fun=roulette_selection, cross_fun=two_point_crossover,
              max_no_change = 500, pop_inlcude_zero = False, verbose = True,
              length_fun = edp, black_list = not_primes_bool)
end = datetime.now()

print('\nTotal time: {}'.format((end-start).total_seconds()))

Iter 0, ItNoChange 0, Best 445901367.318844
Iter 1000, ItNoChange 31, Best 444264042.46257126
Iter 2000, ItNoChange 31, Best 444250435.2917634
Iter 3000, ItNoChange 4, Best 444222143.5938322
Iter 4000, ItNoChange 72, Best 444184727.3995493
Iter 5000, ItNoChange 137, Best 444175067.2122146

Total time: 4313.435948


In [87]:
no_clust_route

array([101535,  22710,  72327, ..., 174326,  39315,  46604])

In [88]:
cities = np.genfromtxt('cities.csv', delimiter=',', skip_header = 1)

In [90]:
total_length_loop(no_clust_route, cities)

440272671.9725418

In [91]:
edp(no_clust_route, cities, not_primes_bool)

444162148.6409465